# My Rating

### 1 Prepare data
##### 1.1 Init Title Base

In [1]:
import pandas as pd
import numpy as np

TITLE_TYPE_FILTER = ['movie', 'tvSeries', 'tvMovie', 'tvMiniSeries', 'tvEpisode']
#FILTER_GENRES = ['Animation', 'Game-Show', 'Music', 'Musical', 'News', 'Reality-TV', 'Short', 'Sport', 'Talk-Show', 'Documentary']
FILTER_GENRES = []
START_YEAR = 1927
END_YEAR = 2026
MIN_NUM_VOTES = 100

def filter_genres(genres):
    s = str(genres).split(',')
    for f in FILTER_GENRES:
        if f in s:
            return 'OUT'
    return genres

ratings = pd.read_csv('data/imdb/title.ratings.tsv.gz', sep='\t', compression='gzip', low_memory=False)
basics = pd.read_csv('data/imdb/title.basics.tsv.gz', sep='\t', compression='gzip', low_memory=False)
title_base = pd.merge(ratings, basics, on='tconst', how='inner')

title_base = title_base[title_base['titleType'].isin(TITLE_TYPE_FILTER)]
title_base = title_base[title_base['numVotes'] > MIN_NUM_VOTES]

title_base = title_base[title_base['startYear'] != '\\N']
title_base['startYear'] = title_base['startYear'].astype('int')
title_base = title_base[(title_base['startYear'] > START_YEAR) & (title_base['startYear'] < END_YEAR)]
title_base['endYear'] = title_base['endYear'].where(title_base['endYear'] != '\\N', title_base['startYear'])
title_base['endYear'] = title_base['endYear'].astype('int')

title_base['runtimeMinutes'] = title_base['runtimeMinutes'].where(title_base['runtimeMinutes'] != '\\N', 0).astype('int')
title_base = title_base[((title_base['titleType'] == 'tvEpisode') & (title_base['runtimeMinutes'] < 151)) | (title_base['runtimeMinutes'] == 0) | (title_base['runtimeMinutes'] > 15)]

title_base['genres'] = title_base['genres'].replace("\\N", np.nan)

title_base = title_base.loc[title_base.groupby(['primaryTitle', 'startYear'])['numVotes'].idxmax()]

COLUMN_NAMES = {'averageRating': 'imdb_rating', 
                'numVotes': 'imdb_num_votes', 
                'titleType': 'title_type', 
                'startYear': 'start_year', 
                'endYear': 'end_year', 
                'primaryTitle': 'title', 
                'runtimeMinutes': 'runtime_minutes'}
title_base = title_base.rename(columns=COLUMN_NAMES)

title_base = title_base[['tconst', 
                           'imdb_rating', 
                           'imdb_num_votes', 
                           'title_type', 
                           'title', 
                           'start_year', 
                           'end_year', 
                           'runtime_minutes', 
                           'genres', 
                           'num_genres']]

KeyError: "['num_genres'] not in index"

In [2]:
title_base.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 344191 entries, 1188755 to 1069369
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           344191 non-null  object 
 1   imdb_rating      344191 non-null  float64
 2   imdb_num_votes   344191 non-null  int64  
 3   title_type       344191 non-null  object 
 4   title            344191 non-null  object 
 5   start_year       344191 non-null  int64  
 6   end_year         344191 non-null  int64  
 7   runtime_minutes  344191 non-null  int64  
 8   genres           343816 non-null  object 
 9   num_genres       344191 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 110.0 MB


#### merge parent titles

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3056819 entries, 0 to 3056818
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   tconst         object
 1   tconst_parent  object
 2   parent_title   object
 3   season         object
 4   episode        object
 5   parent_genres  object
dtypes: object(6)
memory usage: 1.1 GB


,tconst,imdb_rating,imdb_num_votes,title_type,title,start_year,end_year,runtime_minutes,genres,num_genres,tconst_parent,parent_title,season,episode
0,tt33372836,8.8,156,tvEpisode,"and """"",2024,2024,23,"Animation,Comedy,Drama",3,tt21975478,Days with My Stepsister,1,12
1,tt1699720,6.9,266,movie,!Women Art Revolution,2010,2010,83,Documentary,1,,,0,0
2,tt7441016,7.8,308,tvEpisode,#1 Fan,2017,2017,41,"Action,Adventure,Crime",3,tt3560084,NCIS: New Orleans,4,2
3,tt18566094,6.7,930,tvSeries,#1 Happy Family USA,2025,2025,80,"Animation,Comedy",2,,,0,0
4,tt9117124,8.0,381,tvEpisode,#1 Mom,2020,2020,0,"Drama,Sport",2,tt9117054,Spinning Out,1,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344186,tt4422510,3.5,123,movie,"Üç, Iki, Bir... Kestik!",2014,2014,92,Comedy,1,,,0,0
344187,tt22750680,7.7,237,tvEpisode,Üçgen,2022,2022,30,"Action,Comedy",2,tt14344266,Running Errands,3,1
344188,tt27331011,4.7,641,movie,Üçlü Pürüz,2023,2023,105,Comedy,1,,,0,0
344189,tt3509772,5.6,125,movie,ärtico,2014,2014,78,Drama,1,,,0,0


In [19]:
merged_base[merge_base['title'] == '\\N']

NameError: name 'merge_base' is not defined

In [ ]:
merged_base['title_type_id'] = merged_base['title_type'].apply(lambda x: 0 if x == 'movie' else (1 if x == 'tvSeries' else (2 if x == 'tvMiniSeries' else (3 if x == 'tvMovie' else (4 if x == 'tvEpisode' else -1)))))

In [32]:
merged_base[merged_base['genres'] == '']

,tconst,imdb_rating,imdb_num_votes,title_type,title,start_year,end_year,runtime_minutes,genres,num_genres,title_type_id,tconst_parent,parent_title,season,episode,id,parent_id
217153,tt37421468,9.3,157,movie,1st Day 1st Show,2025,2025,0,,1,0,NaN,,0,0,37421468,0
205769,tt32195896,3.7,137,movie,3ALMASHI,2024,2024,100,,1,0,NaN,,0,0,32195896,0
44406,tt0156249,5.7,101,movie,4 Screws and an Anal,1994,1994,0,,1,0,NaN,,0,0,156249,0
215225,tt36329718,4.5,1076,movie,916 Kunjuttan,2025,2025,116,,1,0,NaN,,0,0,36329718,0
203536,tt31438256,6.2,771,tvMovie,A Carol for Two,2024,2024,90,,1,3,NaN,,0,0,31438256,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159707,tt16731494,9.2,109,tvSeries,XTale,2018,2020,196,,1,1,NaN,,0,0,16731494,0
215907,tt36718044,8.0,405,movie,Yaman Kattalai,2025,2025,120,,1,0,NaN,,0,0,36718044,0
57840,tt0311943,6.9,2139,tvEpisode,You'll Get Over It,2002,2002,86,,1,4,NaN,,0,0,311943,0
257245,tt8208102,5.4,197,movie,Zhagaram,2019,2019,113,,1,0,NaN,,0,0,8208102,0


In [3]:
full_df = merged_base.copy()

#### Add Genres Grid

In [4]:
genres_set = {'Action', 'Adult', 'Adventure', 'Biography', 'Comedy', 'Crime',
              'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History',
              'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'}
# Create a column for each genre
for genre in genres_set:
    full_df[f'is_{genre.lower().replace("-", "_")}'] = full_df['genres'].apply(lambda x: 1 if isinstance(x, str) and genre in str(x).split(',') else 0)

## Directors and Writers

In [5]:
name = pd.read_csv('data/imdb/name.basics.tsv.gz', sep='\t', compression='gzip', low_memory=False)
crew = pd.read_csv('data/imdb/title.crew.tsv.gz', sep='\t', compression='gzip', low_memory=False)

In [6]:
crew['directors_list'] = crew['directors'].apply(lambda x: x.split(','))
crew_exploded = crew[['tconst', 'directors_list']].explode('directors_list', ignore_index=True)
crew_exploded = pd.merge(crew_exploded, name, left_on='directors_list', right_on='nconst', how='inner')
directors = crew_exploded[['tconst', 'nconst', 'directors_list', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession']]
directors = directors.rename(columns={'directors_list': 'director', 'primaryName': 'director_name', 'birthYear': 'director_birthYear', 'deathYear': 'director_deathYear', 'primaryProfession': 'director_primaryProfession'})
directors['director_birthYear'] = directors['director_birthYear'].apply(lambda x: '0' if x == '\\N' else x)
directors['director_deathYear'] = directors['director_deathYear'].apply(lambda x: '0' if x == '\\N' else x)
directors['director_birthYear'] = directors['director_birthYear'].fillna('0').astype('int')
directors['director_deathYear'] = directors['director_deathYear'].fillna('0').astype('int')
directors['director_id'] = directors['director'].apply(lambda x: int(x.split('nm')[1])).astype('int')

In [7]:
crew['writers_list'] = crew['writers'].apply(lambda x: x.split(','))
writers_exploded = crew[['tconst', 'writers_list']].explode('writers_list', ignore_index=True)
writers_exploded = pd.merge(writers_exploded, name, left_on='writers_list', right_on='nconst', how='inner')
writers = writers_exploded[['tconst', 'nconst','writers_list', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession']]
writers = writers.rename(columns={'writers_list': 'writer', 'nconst': 'writer_nconst', 'primaryName': 'writer_name', 'birthYear': 'writer_birthYear', 'deathYear': 'writer_deathYear', 'primaryProfession': 'writer_primaryProfession'})
writers['writer_birthYear'] = writers['writer_birthYear'].apply(lambda x: '0' if x == '\\N' else x)
writers['writer_deathYear'] = writers['writer_deathYear'].apply(lambda x: '0' if x == '\\N' else x)
writers['writer_birthYear'] = writers['writer_birthYear'].fillna('0').astype('int')
writers['writer_deathYear'] = writers['writer_deathYear'].fillna('0').astype('int')
writers['writer_id'] = writers['writer'].apply(lambda x: int(x.split('nm')[1])).astype('int')

In [8]:
directors_merged = pd.merge(full_df, directors, on='tconst', how='inner').drop_duplicates()
writers_merged = pd.merge(full_df, writers, on='tconst', how='inner').drop_duplicates()

## Directors Analysis

In [9]:
directors_agg = directors_merged.copy()

In [10]:
directors_count = directors_agg[directors_agg['title_type'] != 'tvSeries'].groupby(['director'])['tconst'].count().rename('director_title_count')
directors_num_votes = directors_agg[directors_agg['title_type'] != 'tvSeries'].groupby(['director'])['imdb_num_votes'].sum().rename('director_num_votes')

In [11]:
directors_rating = (
    directors_agg[directors_agg['title_type'] != 'tvSeries']
    .sort_values(by=['director', 'director_name', 'imdb_rating'], ascending=[True, True, False])
    .groupby(['director', 'director_name'])
    .head(3)
)
directors_rating_agg = directors_rating.groupby(['director', 'director_name'])['imdb_rating'].mean().rename('director_rating')

In [12]:
directors_agg = directors_agg.merge(directors_count, on='director', how='left')
directors_agg = directors_agg.merge(directors_num_votes, on='director', how='left')
directors_agg = directors_agg.merge(directors_rating_agg, on='director', how='left')

In [13]:
genres_cols = ['is_adventure', 'is_action', 'is_film_noir', 'is_sci_fi',
 'is_war', 'is_drama', 'is_thriller', 'is_mystery', 'is_horror',
 'is_crime', 'is_comedy', 'is_family', 'is_fantasy', 'is_history',
 'is_biography', 'is_western', 'is_romance']

for g in genres_cols:
    tmp = directors_agg[directors_agg[g] == 1].groupby('director')['tconst'].count().rename(f'director_count_{g[3:]}_titles')
    directors_agg = directors_agg.merge(tmp, on='director', how='left')
    tmp = directors_agg[directors_agg[g] == 1].groupby('director')['imdb_rating'].mean().rename(f'director_avg_{g[3:]}_rating')
    directors_agg = directors_agg.merge(tmp, on='director', how='left')
    

directors_agg.describe().T


,count,mean,std,min,25%,50%,75%,max
imdb_rating,351312.0,6.704413,1.351722,1.00,5.900,7.000000,7.700000,10.0
imdb_num_votes,351312.0,9906.314259,61131.287743,101.00,198.000,469.000000,1982.000000,3078052.0
start_year,351312.0,2001.734102,20.630657,1928.00,1991.000,2009.000000,2017.000000,2025.0
end_year,351312.0,2002.839166,20.350620,1928.00,1993.000,2010.000000,2018.000000,2026.0
runtime_minutes,351312.0,63.351406,46.566405,0.00,30.000,60.000000,92.000000,3900.0
...,...,...,...,...,...,...,...,...
director_avg_biography_rating,65734.0,6.892039,0.905180,1.00,6.400,6.950000,7.540000,9.4
director_count_western_titles,54783.0,10.662852,21.762767,1.00,1.000,3.000000,10.000000,204.0
director_avg_western_rating,54783.0,6.970485,0.933844,1.85,6.475,7.300000,7.600000,8.8
director_count_romance_titles,213941.0,12.874844,34.111574,1.00,1.000,4.000000,10.000000,321.0


In [14]:
columns_to_fill = ['director_title_count',
'director_num_votes', 'director_rating',
'director_count_adventure_titles', 'director_avg_adventure_rating',
'director_count_action_titles', 'director_avg_action_rating',
'director_count_film_noir_titles', 'director_avg_film_noir_rating',
'director_count_sci_fi_titles', 'director_avg_sci_fi_rating',
'director_count_war_titles', 'director_avg_war_rating',
'director_count_drama_titles', 'director_avg_drama_rating',
'director_count_thriller_titles', 'director_avg_thriller_rating',
'director_count_mystery_titles', 'director_avg_mystery_rating',
'director_count_horror_titles', 'director_avg_horror_rating',
'director_count_crime_titles', 'director_avg_crime_rating',
'director_count_comedy_titles', 'director_avg_comedy_rating',
'director_count_family_titles', 'director_avg_family_rating',
'director_count_fantasy_titles', 'director_avg_fantasy_rating',
'director_count_history_titles', 'director_avg_history_rating',
'director_count_biography_titles', 'director_avg_biography_rating',
'director_count_western_titles', 'director_avg_western_rating',
'director_count_romance_titles', 'director_avg_romance_rating']

directors_agg[columns_to_fill] = directors_agg[columns_to_fill].fillna(0)

In [15]:
directors_agg['director_specific_rating'] = (directors_agg['is_adventure'] * directors_agg['director_avg_adventure_rating'] + 
    directors_agg['is_action'] * directors_agg['director_avg_action_rating'] + 
    directors_agg['is_film_noir'] * directors_agg['director_avg_film_noir_rating'] + 
    directors_agg['is_sci_fi'] * directors_agg['director_avg_sci_fi_rating'] + 
    directors_agg['is_war'] * directors_agg['director_avg_war_rating'] + 
    directors_agg['is_drama'] * directors_agg['director_avg_drama_rating'] + 
    directors_agg['is_thriller'] * directors_agg['director_avg_thriller_rating'] + 
    directors_agg['is_mystery'] * directors_agg['director_avg_mystery_rating'] + 
    directors_agg['is_horror'] * directors_agg['director_avg_horror_rating'] + 
    directors_agg['is_crime'] * directors_agg['director_avg_crime_rating'] + 
    directors_agg['is_comedy'] * directors_agg['director_avg_comedy_rating'] + 
    directors_agg['is_family'] * directors_agg['director_avg_family_rating'] + 
    directors_agg['is_fantasy'] * directors_agg['director_avg_fantasy_rating'] + 
    directors_agg['is_history'] * directors_agg['director_avg_history_rating'] + 
    directors_agg['is_biography'] * directors_agg['director_avg_biography_rating'] + 
    directors_agg['is_western'] * directors_agg['director_avg_western_rating'] + 
    directors_agg['is_romance'] * directors_agg['director_avg_romance_rating'])/ directors_agg['num_genres']
        
            

## Writers Analysis

In [16]:
writers_agg = writers_merged.copy()
writers_count = writers_agg[writers_agg['title_type'] != 'tvSeries'].groupby(['writer'])[
    'tconst'].count().rename('writer_title_count')
writers_num_votes = writers_agg[writers_agg['title_type'] != 'tvSeries'].groupby(['writer'])[
    'imdb_num_votes'].sum().rename('writer_num_votes')

In [ ]:
# writers_rating = writers_agg.merge(writers_agg[['writer_nconst', 'imdb_rating', 'tconst_parent']], on='writer_nconst', how='inner', suffixes=('', '_y'))
# writers_rating.head(1)
# writers_rating = writers_rating[writers_rating['tconst'] != writers_rating['tconst']][writers_rating['tconst_parent'] != writers_rating['tconst_parent_y']]

In [17]:
writers_rating = (
    writers_agg[writers_agg['title_type'] != 'tvSeries']
    .sort_values(by=['writer', 'writer_name', 'imdb_rating'], ascending=[True, True, False])
    .groupby(['writer', 'writer_name'])
    .head(3)
)
writers_rating_agg = writers_rating.groupby(['writer', 'writer_name'])['imdb_rating'].mean().rename(
    'writer_rating')
writers_agg = writers_agg.merge(writers_count, on='writer', how='left')
writers_agg = writers_agg.merge(writers_num_votes, on='writer', how='left')
writers_agg = writers_agg.merge(writers_rating_agg, on='writer', how='left')

In [18]:
genres_cols = ['is_adventure', 'is_action', 'is_film_noir', 'is_sci_fi',
               'is_war', 'is_drama', 'is_thriller', 'is_mystery', 'is_horror',
               'is_crime', 'is_comedy', 'is_family', 'is_fantasy', 'is_history',
               'is_biography', 'is_western', 'is_romance']

for g in genres_cols:
    tmp = writers_agg[writers_agg[g] == 1].groupby('writer')['tconst'].count().rename(
        f'writer_count_{g[3:]}_titles')
    writers_agg = writers_agg.merge(tmp, on='writer', how='left')
    tmp = writers_agg[writers_agg[g] == 1].groupby('writer')['imdb_rating'].mean().rename(
        f'writer_avg_{g[3:]}_rating')
    writers_agg = writers_agg.merge(tmp, on='writer', how='left')

columns_to_fill = ['writer_title_count',
                   'writer_num_votes', 'writer_rating',
                   'writer_count_adventure_titles', 'writer_avg_adventure_rating',
                   'writer_count_action_titles', 'writer_avg_action_rating',
                   'writer_count_film_noir_titles', 'writer_avg_film_noir_rating',
                   'writer_count_sci_fi_titles', 'writer_avg_sci_fi_rating',
                   'writer_count_war_titles', 'writer_avg_war_rating',
                   'writer_count_drama_titles', 'writer_avg_drama_rating',
                   'writer_count_thriller_titles', 'writer_avg_thriller_rating',
                   'writer_count_mystery_titles', 'writer_avg_mystery_rating',
                   'writer_count_horror_titles', 'writer_avg_horror_rating',
                   'writer_count_crime_titles', 'writer_avg_crime_rating',
                   'writer_count_comedy_titles', 'writer_avg_comedy_rating',
                   'writer_count_family_titles', 'writer_avg_family_rating',
                   'writer_count_fantasy_titles', 'writer_avg_fantasy_rating',
                   'writer_count_history_titles', 'writer_avg_history_rating',
                   'writer_count_biography_titles', 'writer_avg_biography_rating',
                   'writer_count_western_titles', 'writer_avg_western_rating',
                   'writer_count_romance_titles', 'writer_avg_romance_rating']

writers_agg[columns_to_fill] = writers_agg[columns_to_fill].fillna(0)
writers_agg['writer_specific_rating'] = (
            writers_agg['is_adventure'] * writers_agg['writer_avg_adventure_rating'] +
            writers_agg['is_action'] * writers_agg['writer_avg_action_rating'] +
            writers_agg['is_film_noir'] * writers_agg['writer_avg_film_noir_rating'] +
            writers_agg['is_sci_fi'] * writers_agg['writer_avg_sci_fi_rating'] +
            writers_agg['is_war'] * writers_agg['writer_avg_war_rating'] +
            writers_agg['is_drama'] * writers_agg['writer_avg_drama_rating'] +
            writers_agg['is_thriller'] * writers_agg['writer_avg_thriller_rating'] +
            writers_agg['is_mystery'] * writers_agg['writer_avg_mystery_rating'] +
            writers_agg['is_horror'] * writers_agg['writer_avg_horror_rating'] +
            writers_agg['is_crime'] * writers_agg['writer_avg_crime_rating'] +
            writers_agg['is_comedy'] * writers_agg['writer_avg_comedy_rating'] +
            writers_agg['is_family'] * writers_agg['writer_avg_family_rating'] +
            writers_agg['is_fantasy'] * writers_agg['writer_avg_fantasy_rating'] +
            writers_agg['is_history'] * writers_agg['writer_avg_history_rating'] +
            writers_agg['is_biography'] * writers_agg['writer_avg_biography_rating'] +
            writers_agg['is_western'] * writers_agg['writer_avg_western_rating'] +
            writers_agg['is_romance'] * writers_agg['writer_avg_romance_rating']
                                        ) / writers_agg['num_genres']

## Combine Directors and Writers

In [19]:
ddf = directors_agg[['tconst', 'director_id',
'director_name', 'director_birthYear', 'director_deathYear',
'director_primaryProfession', 'director_title_count',
'director_num_votes', 'director_rating',
'director_count_adventure_titles', 'director_avg_adventure_rating',
'director_count_action_titles', 'director_avg_action_rating',
'director_count_film_noir_titles', 'director_avg_film_noir_rating',
'director_count_sci_fi_titles', 'director_avg_sci_fi_rating',
'director_count_war_titles', 'director_avg_war_rating',
'director_count_drama_titles', 'director_avg_drama_rating',
'director_count_thriller_titles', 'director_avg_thriller_rating',
'director_count_mystery_titles', 'director_avg_mystery_rating',
'director_count_horror_titles', 'director_avg_horror_rating',
'director_count_crime_titles', 'director_avg_crime_rating',
'director_count_comedy_titles', 'director_avg_comedy_rating',
'director_count_family_titles', 'director_avg_family_rating',
'director_count_fantasy_titles', 'director_avg_fantasy_rating',
'director_count_history_titles', 'director_avg_history_rating',
'director_count_biography_titles', 'director_avg_biography_rating',
'director_count_western_titles', 'director_avg_western_rating',
'director_count_romance_titles', 'director_avg_romance_rating',
'director_specific_rating']]

In [20]:
wdf = writers_agg[['tconst', 'writer_id','writer_name', 'writer_birthYear', 'writer_deathYear',
                   'writer_primaryProfession', 'writer_title_count', 'writer_num_votes', 'writer_rating', 
                   'writer_count_adventure_titles', 'writer_avg_adventure_rating', 
                   'writer_count_action_titles', 'writer_avg_action_rating', 
                   'writer_count_film_noir_titles', 'writer_avg_film_noir_rating', 
                   'writer_count_sci_fi_titles','writer_avg_sci_fi_rating', 
                   'writer_count_war_titles','writer_avg_war_rating', 
                   'writer_count_drama_titles','writer_avg_drama_rating', 
                   'writer_count_thriller_titles','writer_avg_thriller_rating', 
                   'writer_count_mystery_titles','writer_avg_mystery_rating', 
                   'writer_count_horror_titles','writer_avg_horror_rating', 
                   'writer_count_crime_titles','writer_avg_crime_rating', 
                   'writer_count_comedy_titles','writer_avg_comedy_rating', 
                   'writer_count_family_titles','writer_avg_family_rating', 
                   'writer_count_fantasy_titles','writer_avg_fantasy_rating', 
                   'writer_count_history_titles','writer_avg_history_rating', 
                   'writer_count_biography_titles','writer_avg_biography_rating', 
                   'writer_count_western_titles','writer_avg_western_rating', 
                   'writer_count_romance_titles','writer_avg_romance_rating', 
                   'writer_specific_rating']]

In [21]:
full_df_my = full_df.copy()
full_df_my = full_df_my.merge(ddf, on='tconst', how='left', suffixes=('', '_director'))
full_df_my = full_df_my.merge(wdf, on='tconst', how='left', suffixes=('', '_writer'))

In [25]:
mdf = pd.read_csv('data/imdb/my_rating.csv')
mdf = mdf[['Const', 'Your Rating']].rename(columns={'Const': 'tconst', 'Your Rating': 'my_rating'})
full_df_my = full_df_my.merge(mdf, on='tconst', how='left', suffixes=('', '_level1'))
full_df_my = full_df_my.merge(
    mdf,
    left_on='tconst_parent',
    right_on='tconst',
    how='left',
    suffixes=('', '_level2')
)
full_df_my["my_rating"] = full_df_my["my_rating"].combine_first(full_df_my["my_rating_level2"])

In [30]:
full_df_my['season'] = full_df_my['season'].apply(lambda x: '0' if x == '\\N' else x).fillna('0').astype('int')
full_df_my['episode'] = full_df_my['episode'].apply(lambda x: '0' if x == '\\N' else x).fillna('0').astype('int')

In [32]:
full_df_my.to_csv(f'data/imdb/full_df_my.csv', index=False)

## Load Full Data

In [3]:
import pandas as pd
full_df_my = pd.read_csv(f'data/imdb/full_df_my.csv', low_memory=False)

In [8]:
','.join(full_df_my.columns.to_list())

'tconst,imdb_rating,imdb_num_votes,title_type,title,start_year,end_year,runtime_minutes,genres,num_genres,title_type_id,tconst_parent,parent_title,season,episode,id,parent_id,is_comedy,is_fantasy,is_history,is_romance,is_film_noir,is_adventure,is_adult,is_thriller,is_family,is_action,is_drama,is_crime,is_horror,is_western,is_mystery,is_biography,is_war,is_sci_fi,director_id,director_name,director_birthYear,director_deathYear,director_primaryProfession,director_title_count,director_num_votes,director_rating,director_count_adventure_titles,director_avg_adventure_rating,director_count_action_titles,director_avg_action_rating,director_count_film_noir_titles,director_avg_film_noir_rating,director_count_adult_titles,director_avg_adult_rating,director_count_sci_fi_titles,director_avg_sci_fi_rating,director_count_war_titles,director_avg_war_rating,director_count_drama_titles,director_avg_drama_rating,director_count_thriller_titles,director_avg_thriller_rating,director_count_mystery_titles,dire

In [26]:
full_df_my.groupby(['parent_id', 'parent_title']).agg(tconst_count=('tconst', 'count')).sort_values(by='tconst_count', ascending=False)

,,tconst_count
parent_id,parent_title,
56751,Doctor Who,3080
47736,Gunsmoke,2561
203259,Law & Order: Special Victims Unit,2071
898266,The Big Bang Theory,1986
1091909,Murdoch Mysteries,1833
...,...,...
66631,Bearcats!,1
21332800,School Tales the Series,1
469009,Turkish for Beginners,1


In [27]:
full_df_my[full_df_my['parent_id'] == 898266].head(100)

,tconst,imdb_rating,imdb_num_votes,title_type,title,start_year,end_year,runtime_minutes,genres,num_genres,...,writer_count_biography_titles,writer_avg_biography_rating,writer_count_western_titles,writer_avg_western_rating,writer_count_romance_titles,writer_avg_romance_rating,writer_specific_rating,my_rating,tconst_level2,my_rating_level2
2734521,tt0775431,8.1,7930,tvEpisode,Pilot,2007,2007,23,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,887.0,7.514431,7.520439,10.0,tt0898266,10.0
2734522,tt0775431,8.1,7930,tvEpisode,Pilot,2007,2007,23,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,341.0,7.692375,7.686215,10.0,tt0898266,10.0
4391179,tt4995126,7.4,3403,tvEpisode,The 2003 Approximation,2015,2015,19,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,887.0,7.514431,7.520439,10.0,tt0898266,10.0
4391180,tt4995126,7.4,3403,tvEpisode,The 2003 Approximation,2015,2015,19,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,341.0,7.692375,7.686215,10.0,tt0898266,10.0
4391181,tt4995126,7.4,3403,tvEpisode,The 2003 Approximation,2015,2015,19,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,219.0,7.806393,7.747105,10.0,tt0898266,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4416191,tt4962326,7.7,3613,tvEpisode,The Bachelor Party Corrosion,2015,2015,20,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,164.0,7.704268,7.684921,10.0,tt0898266,10.0
4416192,tt4962326,7.7,3613,tvEpisode,The Bachelor Party Corrosion,2015,2015,20,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,78.0,7.753846,7.732779,10.0,tt0898266,10.0
4416193,tt4962326,7.7,3613,tvEpisode,The Bachelor Party Corrosion,2015,2015,20,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,148.0,7.686486,7.595910,10.0,tt0898266,10.0
4416194,tt4962326,7.7,3613,tvEpisode,The Bachelor Party Corrosion,2015,2015,20,"Comedy,Romance",2,...,0.0,0.0,0.0,0.0,82.0,7.530488,7.536410,10.0,tt0898266,10.0


In [ ]:
full_df_train = full_df_my[['imdb_rating', 'imdb_num_votes', 'title_type_id', 'parent_id', 'start_year','end_year','runtime_minutes','num_genres','season','episode', 'is_comedy', 'is_fantasy', 'is_history', 'is_romance', 'is_adventure', 'is_thriller', 'is_family', 'is_action', 'is_drama', 'is_crime', 'is_horror', 'is_western', 'is_mystery', 'is_biography', 'is_war', 'is_sci_fi',
'director_id','director_birthYear','director_deathYear', 'director_title_count','director_num_votes','director_rating',
director_count_adventure_titles, director_avg_adventure_rating, 
                            director_count_action_titles, director_avg_action_rating, director_count_film_noir_titles, director_avg_film_noir_rating, director_count_adult_titles, director_avg_adult_rating, director_count_sci_fi_titles, director_avg_sci_fi_rating, director_count_war_titles, director_avg_war_rating, director_count_drama_titles, director_avg_drama_rating, director_count_thriller_titles, director_avg_thriller_rating, director_count_mystery_titles, director_avg_mystery_rating, director_count_horror_titles, director_avg_horror_rating, director_count_crime_titles, director_avg_crime_rating, director_count_comedy_titles, director_avg_comedy_rating, director_count_family_titles, director_avg_family_rating, director_count_fantasy_titles, director_avg_fantasy_rating, director_count_history_titles, director_avg_history_rating, director_count_biography_titles, director_avg_biography_rating, director_count_western_titles, director_avg_western_rating, director_count_romance_titles, director_avg_romance_rating, director_specific_rating, writer_id, writer_name, writer_birthYear, writer_deathYear, writer_primaryProfession, writer_title_count, writer_num_votes, writer_rating, writer_count_adventure_titles, writer_avg_adventure_rating, writer_count_action_titles, writer_avg_action_rating, writer_count_film_noir_titles, writer_avg_film_noir_rating, writer_count_adult_titles, writer_avg_adult_rating, writer_count_sci_fi_titles, writer_avg_sci_fi_rating, writer_count_war_titles, writer_avg_war_rating, writer_count_drama_titles, writer_avg_drama_rating, writer_count_thriller_titles, writer_avg_thriller_rating, writer_count_mystery_titles, writer_avg_mystery_rating, writer_count_horror_titles, writer_avg_horror_rating, writer_count_crime_titles, writer_avg_crime_rating, writer_count_comedy_titles, writer_avg_comedy_rating, writer_count_family_titles, writer_avg_family_rating, writer_count_fantasy_titles, writer_avg_fantasy_rating, writer_count_history_titles, writer_avg_history_rating, writer_count_biography_titles, writer_avg_biography_rating, writer_count_western_titles, writer_avg_western_rating, writer_count_romance_titles, writer_avg_romance_rating, writer_specific_rating, my_rating, tconst_level2, my_rating_level2
                            
                            ]]

In [ ]:
full_df_snapshot = full_df_my

In [12]:
tsv_2024 = pd.read_csv(f'data/imdb/movies_2024.tsv', low_memory=False, delimiter='\t', on_bad_lines="skip")

In [15]:
tsv_2024.head(100)

,id,imdb,original_language,overview,popularity,release_date,vote_average,vote_count,production_companies_names,production_companies_id,production_companies_country,budget,revenue
0,209506,tt0251151,mr,A story centered on a Nagya who is a drummer h...,0.5884,1977-07-23,5.5,4,NaN,NaN,NaN,0.0,0.0
1,187716,tt0287075,en,"Filmed in 1999, Universal Groove explores 90s ...",0.1971,2024-02-25,10.0,1,IMPACT MEDIA,221814,NaN,0.0,0.0
2,507241,tt0327785,en,When top hitman Joe Flood is diagnosed with a ...,6.5133,2024-09-12,6.381,447,Dogbone Entertainment|Mad Chance|Endurance Med...,183234|1757|89171|175316|253068|253069,US|US|US|US|GB|ES,30000000.0,5928351.0
3,1212149,tt10009608,es,"Ximena, a successful psychiatrist, receives an...",0.8072,2024-09-21,8.0,2,b-mount film|Storyboard Media|Morocha Films|Mu...,156828|108416|2204|79782,JP|CL|AR|AR,0.0,0.0
4,1043758,tt10013512,en,Two estranged cousins who never knew each othe...,2.5838,2024-10-04,6.2,13,Maxfilms,115461,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1204906,tt12234316,en,Private investigator Mort Madison's life begin...,0.4586,2024-02-01,2.0,1,Illusion Islands,186161,NaN,0.0,0.0
96,1219348,tt12246076,en,A girl is drugged and assaulted by her boyfrie...,0.6986,2024-01-09,5.333,6,PhilaDreams Films|Safier Entertainment|Lux Ang...,212068|87066|164544|216301,|US||,0.0,0.0
97,607338,tt12276028,en,When Russian workers in Bernie’s house turn ou...,1.1975,2024-05-16,4.8,39,Kaap Holland Film,67254,NL,0.0,0.0
98,1316080,tt12297834,en,"In the dead of night, Charlotte, a young girl,...",0.1474,2024-08-26,7.0,1,NaN,NaN,NaN,0.0,0.0


In [26]:
full_df_my = full_df_my.drop(columns=['tconst', 'tconst_parent', 'title', 'genres', 'parent_title', 'director_name', 'writer_name', 'my_rating_level2', 'tconst_level2', 'is_adult', 'is_film_noir'])

In [9]:
professions = (
    full_df_my['director_primaryProfession']
    .dropna()
    .str.split(',')
    .explode()
    .drop_duplicates()
    .reset_index(drop=True)
)

print(professions)


0                         actor
1                      director
2                        writer
3            assistant_director
4                 miscellaneous
5                      producer
6             camera_department
7                       actress
8              music_department
9                visual_effects
10                       editor
11         editorial_department
12              cinematographer
13                     composer
14           production_manager
15               art_department
16             casting_director
17            script_department
18          production_designer
19                       stunts
20             costume_designer
21           costume_department
22              special_effects
23           make_up_department
24                   soundtrack
25              archive_footage
26                 art_director
27             sound_department
28         animation_department
29           casting_department
30                    executive
31      

## Directors or Writers Results

In [30]:
full_df_my.head(100)

,tconst,imdb_rating,imdb_num_votes,title_type,title,start_year,end_year,runtime_minutes,genres,num_genres,...,writer_avg_fantasy_rating,writer_count_history_titles,writer_avg_history_rating,writer_count_biography_titles,writer_avg_biography_rating,writer_count_western_titles,writer_avg_western_rating,writer_count_romance_titles,writer_avg_romance_rating,writer_specific_rating
9,tt28789294,9.7,282,tvEpisode,#1.1,2023,2023,45,"Comedy,Drama,Romance",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,tt28761646,9.5,323,tvEpisode,#BabbarPajiWapasAao,2023,2023,0,Comedy,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,8.877778
37,tt28761646,9.5,323,tvEpisode,#BabbarPajiWapasAao,2023,2023,0,Comedy,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.90,8.877778
129,tt36958970,7.6,119,tvEpisode,#Bunjeejumping,2025,2025,0,"Comedy,Drama,Romance",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,tt36958967,7.6,110,tvEpisode,#CloudNine,2025,2025,0,"Comedy,Drama,Romance",3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86217,tt29293451,7.6,107,tvEpisode,A Storm is Brewing,2023,2023,49,"Drama,Thriller",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7.923077
86218,tt29293451,7.6,107,tvEpisode,A Storm is Brewing,2023,2023,49,"Drama,Thriller",2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,8.042857
86247,tt9187714,6.9,207,tvMiniSeries,A Story to Read When You First Fall in Love,2019,2019,67,"Comedy,Drama,Romance",3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.90,6.900000
86248,tt9187714,6.9,207,tvMiniSeries,A Story to Read When You First Fall in Love,2019,2019,67,"Comedy,Drama,Romance",3,...,5.9,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.75,6.705556


In [30]:
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from feature_engine.encoding import OrdinalEncoder
from feature_engine.scaling import MeanNormalizationScaler
X = full_df_train.drop(columns=['my_rating'])
y = full_df_train['my_rating']

categorical_cols = X.select_dtypes(include=['object']).columns.to_list()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.to_list()

transformers = [
    ArbitraryNumberImputer(arbitrary_number=0, variables=numerical_cols),
    MeanNormalizationScaler(variables=numerical_cols),
    CategoricalImputer(imputation_method='missing', variables=categorical_cols),
    OrdinalEncoder(encoding_method='ordered', variables=categorical_cols)
]

for t in transformers:
    X = t.fit_transform(X, y)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [1]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
print(f'Test score: {rf.score(X_test, y_test)}')

NameError: name 'X_train' is not defined

In [35]:
X1 = full_df_test.drop(columns=["my_rating"], axis=1)
y1 = full_df_test["my_rating"]
for t in transformers:
    X = t.fit_transform(X1, y1)

ValueError: Some of the variables in the dataset contain NaN. Check and remove those before using this transformer.

In [ ]:
rf.predict(X1.head(3))

In [ ]:
full_df['prediction'] = rf.predict(X)

In [ ]:
#1024588
tmp = full_df_test[full_df_test['imdb_num_votes'] > 1000]
tmp = tmp[tmp['title_type'] == 'tvEpisode']


tmp = (tmp.sort_values(by=['writer_specific_rating',  'imdb_rating'], ascending=[False, False])[['id', 'tconst', 'writer_name', 'title', 'parent_title','start_year', 'genres', 'season', 'episode','writer_specific_rating',  'imdb_rating', 'imdb_num_votes', 'runtime_minutes']]
 .groupby(['writer_name', 'parent_title'])
 .head(3))
tmp

## Principals

In [ ]:
principals = pd.read_csv('data/imdb/title.principals.tsv.gz', sep='\t', compression='gzip', low_memory=False)
principals.merge(name, on='nconst', how='inner')
principals.head()

In [ ]:
principals[principals['tconst'] == 'tt21151974'].head(100)

In [ ]:
year = 2021
tdf = full_df[full_df['start_year'] == year][['tconst', 'title']].sort_values(by='tconst')
tdf.to_csv(f'data/imdb/titles_{year}.csv', index=False)
tdf.count()